- [Linear Regression](#linear-regression)
- [Logistic Regression](#logistic-regression)

In [2]:
import probpy as pp
import numpy as np
import numba
import random

linear regression
---

In [4]:
%%time
def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]

w = [5, -2, 1, 5] # True underlying model
x = np.random.rand(100, 3) * 10
y = predict(w, x) + pp.normal.sample(mu=0, sigma=1, size=100).reshape(-1)



prior = pp.multivariate_normal.med(mu=np.ones(4) * 0, sigma=np.eye(4) * 5)
likelihood = pp.unilinear.med(sigma=1) # There exist an implementation for linear because it has a conjugate prior

for i in range(100):
    data = (y[i], x[i])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, priors=prior)
    
    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - predict(w_approx, x)).mean(), 
              "True MSE", np.square(y - predict(w, x)).mean())
        print()

Parameter Estimate [4.42357948 0.5674214  2.16259965 0.5283677 ]
Prior MSE 198.5711038274159 True MSE 1.1703343416892766

Parameter Estimate [ 5.1226452  -1.69269041  0.91536957  3.17283315]
Prior MSE 2.0007526776754507 True MSE 1.1703343416892766

Parameter Estimate [ 5.0379815  -1.84005992  0.89049501  4.62061906]
Prior MSE 1.3469901773766002 True MSE 1.1703343416892766

Parameter Estimate [ 5.06924187 -1.89299437  0.97664179  4.38430079]
Prior MSE 1.217836305544823 True MSE 1.1703343416892766

Parameter Estimate [ 4.98960191 -1.96094297  0.98871937  5.18124575]
Prior MSE 1.1555839475054817 True MSE 1.1703343416892766

Parameter Estimate [ 4.97401524 -1.96876404  0.98113974  5.23714495]
Prior MSE 1.139769308547774 True MSE 1.1703343416892766

Parameter Estimate [ 4.98065102 -1.98027821  0.98711063  5.15641003]
Prior MSE 1.140450549620556 True MSE 1.1703343416892766

Parameter Estimate [ 5.01976997 -1.93316935  0.96492111  4.90680214]
Prior MSE 1.1443373080585484 True MSE 1.1703343416

logistic regression
---

In [7]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w): # This should be as fast as possible since it is the bottle-neck
    # It is possible to rewrite this loop in pure numpy with some broadcasting magic but 
    # I think this example is more instructive
        
    result = []
    for _w in w:
        prediction = sigmoid(predict(_w, x))
        result.append(pp.normal.p(y - prediction, mu=0, sigma=1.0))
        
    return result

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(50):
    j = random.randint(0, 80)
    data = (y[j: j + 20], x[j: j + 20])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   match_moments_for=pp.multivariate_normal,
                                   batch=50,
                                   samples=5000,
                                   burn_in=500, 
                                   energies=1.0)

    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
              "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
        print()

Parameter Estimate [-0.17428387  0.52075012  0.71061252  0.26381748]
Prior MSE 0.1260172889415909 True MSE 0.020399392079436057

Parameter Estimate [-2.08296048  1.58687583  3.55129168 -1.90589892]
Prior MSE 0.024605646101764286 True MSE 0.020399392079436057

Parameter Estimate [-2.49966395  2.70081893  4.40098167 -2.79841413]
Prior MSE 0.020032873564875847 True MSE 0.020399392079436057

Parameter Estimate [-2.92519776  2.97762299  4.85843405 -2.9953453 ]
Prior MSE 0.02019704002634965 True MSE 0.020399392079436057

Parameter Estimate [-2.95162424  3.40766415  5.06935042 -3.48494753]
Prior MSE 0.0208060435586614 True MSE 0.020399392079436057

CPU times: user 1min 34s, sys: 2min 29s, total: 4min 4s
Wall time: 1min 1s


Logistic Regression + Numpy magic ~ 10x speed-up
---

In [8]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(50):
    j = random.randint(0, 80)
    data = (y[j: j + 20], x[j: j + 20])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   match_moments_for=pp.multivariate_normal,
                                   batch=50,
                                   samples=5000,
                                   burn_in=500, 
                                   energies=1.0)

    if i % 10 == 0:
        w_approx = pp.mode(prior)
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
              "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
        print()

Parameter Estimate [-0.53601026  0.28077979  0.46164084 -0.8330969 ]
Prior MSE 0.0782992400216686 True MSE 0.027590952878289207

Parameter Estimate [-1.96945747  1.46968586  3.37183621 -1.93631875]
Prior MSE 0.02917472725253408 True MSE 0.027590952878289207

Parameter Estimate [-2.54491043  2.14892638  4.12655498 -2.41129613]
Prior MSE 0.026715544705368424 True MSE 0.027590952878289207

Parameter Estimate [-2.78140459  2.54598524  4.3280773  -2.57244346]
Prior MSE 0.0262718273798136 True MSE 0.027590952878289207

Parameter Estimate [-3.13342456  2.65961273  4.56699265 -2.7096384 ]
Prior MSE 0.026869735122740566 True MSE 0.027590952878289207

CPU times: user 14.8 s, sys: 22.8 s, total: 37.6 s
Wall time: 9.41 s


Logistic Regression without moment-matching
---

In [12]:
%%time
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def predict(w, x):
     return x[:, 0] * w[0] + x[:, 1] * w[1] + x[:, 2] * w[2] + w[3]
    
w = [-3, 3, 5, -3] # True underlying model

x = np.random.rand(100, 3)
y = sigmoid(predict(w, x) + pp.normal.sample(mu=0.0, sigma=1.0, size=100).reshape(-1))

# For this we need custom likelihood since there is no conjugate prior

def likelihood(y, x, w):
    return pp.normal.p((y - sigmoid(x @ w[:, :-1, None] + w[:, None, None, -1]).squeeze(axis=2)),
                    mu=0.0, sigma=1.0)
    
    

prior = pp.multivariate_normal.med(mu=np.zeros(4), sigma=np.eye(4) * 5)

for i in range(50):
    j = random.randint(0, 80)
    data = (y[j: j + 20], x[j: j + 20])
    
    prior = pp.parameter_posterior(data, likelihood=likelihood, 
                                   priors=prior, 
                                   batch=5,
                                   samples=1000,
                                   burn_in=100, 
                                   energies=1.0)

    if i % 10 == 0:
        modes = pp.mode(prior) # modes are sorted in order first is largest

        print("Number of modes", len(modes))
        w_approx = modes[0]
        
        print("Parameter Estimate", w_approx)
        
        print("Prior MSE", np.square(y - sigmoid(predict(w_approx, x))).mean(), 
              "True MSE", np.square(y - sigmoid(predict(w, x))).mean())
        print()

Number of modes 5
Parameter Estimate [ 0.25289025  1.07358778  0.22271974 -1.45592379]
Prior MSE 0.08989462571191517 True MSE 0.02557124439040531

Number of modes 3
Parameter Estimate [-1.6441759   1.24321202  2.4961304  -1.67858393]
Prior MSE 0.03433557116653359 True MSE 0.02557124439040531

Number of modes 3
Parameter Estimate [-3.08648632  2.71105106  2.76933978 -1.44667161]
Prior MSE 0.030504307227619015 True MSE 0.02557124439040531

Number of modes 3
Parameter Estimate [-3.40101005  2.76046226  3.64808157 -1.80800666]
Prior MSE 0.027947714687514585 True MSE 0.02557124439040531

Number of modes 3
Parameter Estimate [-3.56976171  2.86013503  3.1125747  -1.88175391]
Prior MSE 0.031942999881937405 True MSE 0.02557124439040531

CPU times: user 14 s, sys: 747 ms, total: 14.8 s
Wall time: 13.9 s
